In [1]:
import sys
sys.path.append('../../')
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision import transforms
import time
from collections import OrderedDict
from torch.utils.data import DataLoader

In [2]:
BATCH_SIZE = 128

USE_JIT = False

device = torch.device('cuda')

In [3]:
mnist = MNIST('../../', transform=transforms.ToTensor())

In [4]:
data_loader = DataLoader(mnist, batch_size=BATCH_SIZE, drop_last=True, num_workers=0)




In [5]:
from Code.Networks import Selector, DynNetwork, OuterWrapper, LSTMWrapper, ReLuWrapper, DummyNeuron, make_SequenceWrapper
from Code.NewNeurons import SeqOnlySpike, CooldownNeuron, OutputNeuron

base_config = {
    'ALPHA': 0,
    'BETA': 0,
    'OFFSET': 2,
    'RESET_ZERO': False,
    'DECODING': 'potential',
    'SPIKE_FN': 'ss'
}

heavyside = {
    **base_config,
    'BETA': 1,
    'OFFSET': 7
}



mem_loop = OrderedDict([
    ('input', 1),
    ('pre_mem', [['input', 'output'], SeqOnlySpike(128, base_config), nn.Linear]),
    ('output', [['pre_mem'], CooldownNeuron(128, heavyside), nn.Linear]),
])

architecture = OrderedDict([
    ('input', 1),
    ('mem_loop', [['input'], make_SequenceWrapper(DynNetwork(mem_loop), USE_JIT), None]),
    ('post_mem', [['mem_loop'], SeqOnlySpike(128, base_config), nn.Linear]),
    ('output', [['post_mem'], OutputNeuron(10, heavyside), nn.Linear]),
])

architecturelstm = OrderedDict([
    ('input', 1),
    ('lstm', [['input'], LSTMWrapper(1, 128), None]),
    ('post_mem', [['lstm'], ReLuWrapper(128), nn.Linear]),
    ('output', [['post_mem'], DummyNeuron(10), nn.Linear]),
])

#TODO: fix output


In [6]:
#144, 150, 137, 150

model = OuterWrapper(DynNetwork(architecture), device, USE_JIT)

#model = OuterWrapper(DynNetwork(architecturelstm), device, USE_JIT)

#with torch.no_grad():
#    model.model.layers.lstm.lstm.bias_hh_l0[:256] += 3



In [7]:
ce = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)#0.000011e-6
#optimizer = optim.SGD(model.parameters(), lr=1e-5)


In [ ]:
start = time.time()
i = 0
sumloss = 0
for k in range(60):
    print('Epoch: ', k)
    for inp, target in data_loader:
        batchstart = time.time()
        x = inp.view(BATCH_SIZE, -1, 1).transpose(0,1).to(device)
        target = target.to(device)
        model.zero_grad()
        outputs, _ = model(x)
        loss = ce(outputs[-1], target)
        #loss = ce(torch.softmax(outputs, 2).view(-1,10), target.expand(784, BATCH_SIZE).reshape(-1))
        #loss = ce(outputs.view(-1,10), target.expand(784, BATCH_SIZE).reshape(-1))
        loss.backward()
        optimizer.step()
        sumloss += loss.item()
        if i%100 == 0:
            acc = (torch.argmax(outputs[-1], 1) == target).float().mean()
            print(loss.item(), sumloss/100, acc.item(), time.time()-batchstart, outputs[-1].var(0).mean().item()) #torch.argmax(outputs[-1], 1).float().var()
            sumloss = 0
        i += 1

print('Total time: ', time.time()-start)


Epoch:  0
2.4230031967163086 0.024230031967163085 0.0703125 1.2127103805541992 0.0


In [ ]:
#model.save('../../models/seq_mnist_lstm2')

In [ ]:
model.model.layers.lstm.lstm.bias_hh_l0[128:256]

In [ ]:
model.model.layers.lstm.lstm.bias_hh_l0[128:256]

In [ ]:
x = mnist[0][0].view(-1, 1, 1).to(device)

In [ ]:
model(x)[0][-1]

In [ ]:
model(x)[0][-1]


In [ ]:
outputs

In [ ]:
torch.argmax(outputs[-1], 1)

In [ ]:
torch.tensor(range(10)).float().var()

In [ ]:
print(model(mnist[0][0].view(-1, 1, 1).to(device))[0][-1])
print(model(mnist[1][0].view(-1, 1, 1).to(device))[0][-1])

In [ ]:


x.shape

In [ ]:
target.expand(784, BATCH_SIZE).reshape(-1)[0:10]

In [ ]:
target.expand(784, BATCH_SIZE).reshape(-1)[64:74]


In [ ]:
torch.softmax(outputs, 2).view(-1,10)[64:74]